<a href="https://colab.research.google.com/github/rabinpandey123/credit_scoring_model_codealpha_task/blob/main/credit_scoring_model_using_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression

In [ ]:
file_path = "/content/a_Dataset_CreditScoring.xlsx - Sheet1.csv"
df =pd.read_csv("/content/a_Dataset_CreditScoring.xlsx - Sheet1.csv")
df.head()

,TARGET,ID,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
0,1,582,3,3,0,4,0.0,5,117,27,...,3.0,92%,21%,2,3,7,21%,4,4,0%
1,1,662,15,9,0,3,1.0,3,14,14,...,1.0,80%,0%,0,0,0,100%,12,0,100%
2,1,805,0,0,0,1,5.0,1,354,7,...,5.0,36%,65%,0,1,1,73%,1,1,53%
3,1,1175,8,5,0,6,1.0,10,16,4,...,3.0,91%,25%,1,1,1,75%,7,1,133%
4,1,1373,3,1,0,9,0.0,8,130,52,...,1.0,125%,0%,0,1,4,14%,3,1,0%


In [ ]:
df.shape

(3000, 30)

In [ ]:
X = df.drop('TARGET', axis=1)   # replace 'Target' with your column name
y = df['TARGET']


for col in X.columns:
    if X[col].dtype == 'object' and X[col].str.contains('[\$,]').any():
        X[col] = X[col].replace(r'[\$,]', '', regex=True).astype(float)


<>:6: SyntaxWarning: invalid escape sequence '\$'
<>:6: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-3546519683.py:6: SyntaxWarning: invalid escape sequence '\$'
  if X[col].dtype == 'object' and X[col].str.contains('[\$,]').any():


In [ ]:
# explore missing values
df.isna().sum()

,0
TARGET,0
ID,0
DerogCnt,0
CollectCnt,0
BanruptcyInd,0
InqCnt06,0
InqTimeLast,188
InqFinanceCnt24,0
TLTimeFirst,0
TLTimeLast,0


In [ ]:
dataset = df.apply(lambda col: col.fillna(col.mean()) if col.dtype in ['float64', 'int64'] else col.fillna(col.mode()[0]))


In [ ]:
# explore missing values post missing value fix
dataset.isna().sum()

,0
TARGET,0
ID,0
DerogCnt,0
CollectCnt,0
BanruptcyInd,0
InqCnt06,0
InqTimeLast,0
InqFinanceCnt24,0
TLTimeFirst,0
TLTimeLast,0


In [ ]:
# 0 = Good Loans
# 1 = Bad loans
dataset['TARGET'].value_counts()

,count
TARGET,
0,2500
1,500


In [ ]:
# Step 4: Encode categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns

if len(categorical_cols) > 0:
    X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)


In [ ]:
df.head(400)

,TARGET,ID,DerogCnt,CollectCnt,BanruptcyInd,InqCnt06,InqTimeLast,InqFinanceCnt24,TLTimeFirst,TLTimeLast,...,TL50UtilCnt,TLBalHCPct,TLSatPct,TLDel3060Cnt24,TLDel90Cnt24,TLDel60CntAll,TLOpenPct,TLBadDerogCnt,TLDel60Cnt24,TLOpen24Pct
0,1,582,3,3,0,4,0.0,5,117,27,...,3.0,92%,21%,2,3,7,21%,4,4,0%
1,1,662,15,9,0,3,1.0,3,14,14,...,1.0,80%,0%,0,0,0,100%,12,0,100%
2,1,805,0,0,0,1,5.0,1,354,7,...,5.0,36%,65%,0,1,1,73%,1,1,53%
3,1,1175,8,5,0,6,1.0,10,16,4,...,3.0,91%,25%,1,1,1,75%,7,1,133%
4,1,1373,3,1,0,9,0.0,8,130,52,...,1.0,125%,0%,0,1,4,14%,3,1,0%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1,103027,0,0,0,0,8.0,0,249,7,...,6.0,74%,83%,1,0,0,92%,0,0,45%
396,1,103045,6,6,0,6,1.0,13,64,11,...,0.0,0%,22%,0,3,6,0%,8,3,0%
397,1,103066,19,17,0,1,1.0,3,84,4,...,1.0,33%,20%,2,0,4,40%,15,1,50%
398,1,103123,0,0,0,4,0.0,8,36,3,...,5.0,78%,50%,2,1,2,63%,1,2,80%


In [ ]:
# Extract rows where target = 0
target_0_rows = df[df['TARGET'] == 0]

# View first few rows
print(target_0_rows)


      TARGET      ID  DerogCnt  CollectCnt  BanruptcyInd  InqCnt06  \
500        0      66         1           1             0         7   
501        0     116         1           1             0         2   
502        0     124         0           0             0         1   
503        0     128         0           0             0         6   
504        0     143         0           0             0         1   
...      ...     ...       ...         ...           ...       ...   
2995       0  126441         0           0             0         2   
2996       0  126448         0           0             0         0   
2997       0  126465         1           1             0         8   
2998       0  126487         4           4             0         2   
2999       0  126503         2           1             1         0   

      InqTimeLast  InqFinanceCnt24  TLTimeFirst  TLTimeLast  ...  TL50UtilCnt  \
500           1.0                4          125           3  ...          4.0 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

print("Preprocessing done ✅")

Preprocessing done ✅


In [ ]:
# Check for NaNs
print(X_train.isnull().sum())
print(X_test.isnull().sum())

# Fill any remaining numeric NaNs
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())


ID                  0
DerogCnt            0
CollectCnt          0
BanruptcyInd        0
InqCnt06            0
                   ..
TLOpen24Pct_91%     0
TLOpen24Pct_92%     0
TLOpen24Pct_93%     0
TLOpen24Pct_94%     0
TLOpen24Pct_95%     0
Length: 471, dtype: int64
ID                  0
DerogCnt            0
CollectCnt          0
BanruptcyInd        0
InqCnt06            0
                   ..
TLOpen24Pct_91%     0
TLOpen24Pct_92%     0
TLOpen24Pct_93%     0
TLOpen24Pct_94%     0
TLOpen24Pct_95%     0
Length: 471, dtype: int64


In [ ]:
classifier =  LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(class_weight='balanced', max_iter=1000)
classifier.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000)

In [ ]:
print(y_train.value_counts())


TARGET
0    2018
1     382
Name: count, dtype: int64


In [ ]:
# Check unique values and counts
print(y_train.value_counts())

# Check what value corresponds to good/bad
print(y_train.unique())


TARGET
0    2018
1     382
Name: count, dtype: int64
[0 1]


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(df['TARGET'])
print(le.classes_)  # Shows the original labels in order


[0 1]


In [ ]:
print(y_train.value_counts())
print(np.unique(y_train))  # see if 1 is actually “Good Loan”


TARGET
0    2018
1     382
Name: count, dtype: int64
[0 1]


In [ ]:
print(confusion_matrix(y_test,y_pred))

[[466  16]
 [ 96  22]]


In [ ]:
print(accuracy_score(y_test, y_pred))

0.8133333333333334


In [ ]:
predictions = classifier.predict_proba(X_test)
predictions

array([[0.47936124, 0.52063876],
       [0.5048361 , 0.4951639 ],
       [0.37075704, 0.62924296],
       ...,
       [0.13686814, 0.86313186],
       [0.5148396 , 0.4851604 ],
       [0.95676229, 0.04323771]])

In [ ]:
# 1️⃣ Take first row
user_input = df.iloc[0].copy()

# 2️⃣ Convert $ / , to float
for col in user_input.index:
    if isinstance(user_input[col], str) and ('$' in user_input[col] or ',' in user_input[col]):
        user_input[col] = float(user_input[col].replace('$','').replace(',',''))

# 3️⃣ Convert to DataFrame and align columns
user_input_df = pd.DataFrame([user_input])
user_input_df = user_input_df.reindex(columns=X_train.columns, fill_value=0)

# 4️⃣ Scale numeric columns
numeric_cols = X_train.select_dtypes(include=['float64','int64']).columns
user_input_df[numeric_cols] = scaler.transform(user_input_df[numeric_cols])

# 5️⃣ Predict class
predicted_class = classifier.predict(user_input_df)[0]

# 6️⃣ Map to label
loan_label = "Good Loan" if predicted_class == 1 else "Bad Loan"

# 7️⃣ Predict probability
predicted_prob = classifier.predict_proba(user_input_df)[0]

print(f"Prediction: {loan_label}")
print(f"Probability (Bad Loan / Good Loan): {predicted_prob}")


Prediction: Good Loan
Probability (Bad Loan / Good Loan): [0.44149782 0.55850218]


In [ ]:
# 1️⃣ Take first row
user_input = df.iloc[1].copy()

# 2️⃣ Convert $ / , to float
for col in user_input.index:
    if isinstance(user_input[col], str) and ('$' in user_input[col] or ',' in user_input[col]):
        user_input[col] = float(user_input[col].replace('$','').replace(',',''))

# 3️⃣ Convert to DataFrame and align columns
user_input_df = pd.DataFrame([user_input])
user_input_df = user_input_df.reindex(columns=X_train.columns, fill_value=0)

# 4️⃣ Scale numeric columns
numeric_cols = X_train.select_dtypes(include=['float64','int64']).columns
user_input_df[numeric_cols] = scaler.transform(user_input_df[numeric_cols])

# 5️⃣ Predict class
predicted_class = classifier.predict(user_input_df)[0]

# 6️⃣ Map to label
loan_label = "Good Loan" if predicted_class == 1 else "Bad Loan"

# 7️⃣ Predict probability
predicted_prob = classifier.predict_proba(user_input_df)[0]

print(f"Prediction: {loan_label}")
print(f"Probability (Bad Loan / Good Loan): {predicted_prob}")


Prediction: Bad Loan
Probability (Bad Loan / Good Loan): [0.50452409 0.49547591]


In [ ]:
# 1️⃣ Take first row
user_input = df.iloc[2].copy()

# 2️⃣ Convert $ / , to float
for col in user_input.index:
    if isinstance(user_input[col], str) and ('$' in user_input[col] or ',' in user_input[col]):
        user_input[col] = float(user_input[col].replace('$','').replace(',',''))

# 3️⃣ Convert to DataFrame and align columns
user_input_df = pd.DataFrame([user_input])
user_input_df = user_input_df.reindex(columns=X_train.columns, fill_value=0)

# 4️⃣ Scale numeric columns
numeric_cols = X_train.select_dtypes(include=['float64','int64']).columns
user_input_df[numeric_cols] = scaler.transform(user_input_df[numeric_cols])

# 5️⃣ Predict class
predicted_class = classifier.predict(user_input_df)[0]

# 6️⃣ Map to label
loan_label = "Good Loan" if predicted_class == 1 else "Bad Loan"

# 7️⃣ Predict probability
predicted_prob = classifier.predict_proba(user_input_df)[0]

print(f"Prediction: {loan_label}")
print(f"Probability (Bad Loan / Good Loan): {predicted_prob}")


Prediction: Bad Loan
Probability (Bad Loan / Good Loan): [0.7976824 0.2023176]


In [ ]:
# 1️⃣ Take a row
user_input = df.iloc[2900].copy()

# 2️⃣ Convert $ / , to float
for col in user_input.index:
    if isinstance(user_input[col], str) and ('$' in user_input[col] or ',' in user_input[col]):
        user_input[col] = float(user_input[col].replace('$','').replace(',',''))

# 3️⃣ Convert to DataFrame and align columns
user_input_df = pd.DataFrame([user_input])
user_input_df = user_input_df.reindex(columns=X_train.columns, fill_value=0)

# 4️⃣ Scale numeric columns
numeric_cols = X_train.select_dtypes(include=['float64','int64']).columns
user_input_df[numeric_cols] = scaler.transform(user_input_df[numeric_cols])

# 5️⃣ Predict class
predicted_class = classifier.predict(user_input_df)[0]

# 6️⃣ Map to label
loan_label = "Good Loan" if predicted_class == 1 else "Bad Loan"

# 7️⃣ Predict probability
predicted_prob = classifier.predict_proba(user_input_df)[0]

print(f"Prediction: {loan_label}")
print(f"Probability (Bad Loan / Good Loan): {predicted_prob}")


Prediction: Good Loan
Probability (Bad Loan / Good Loan): [0.20157558 0.79842442]


In [ ]:
# Loop over first 40 rows
for i in range(2000,2999):
    row = df.iloc[i].copy()  # Take the i-th row

    # Preprocess row (convert $ and commas)
    for col in row.index:
        if isinstance(row[col], str) and ('$' in row[col] or ',' in row[col]):
            row[col] = float(row[col].replace('$','').replace(',',''))

    # Convert to DataFrame and align columns
    row_df = pd.DataFrame([row])
    row_df = row_df.reindex(columns=X_train.columns, fill_value=0)

    # Scale numeric columns if using LogisticRegression
    numeric_cols = X_train.select_dtypes(include=['float64','int64']).columns
    row_df[numeric_cols] = scaler.transform(row_df[numeric_cols])

    # Predict class
    predicted_class = classifier.predict(row_df)[0]
    loan_label = "Good Loan" if predicted_class == 1 else "Bad Loan"

    # Predict probabilities
    predicted_prob = classifier.predict_proba(row_df)[0]

    print(f"Row {i} -> Prediction: {loan_label}, Probability (Bad/Good): {predicted_prob}")


Row 2000 -> Prediction: Bad Loan, Probability (Bad/Good): [0.8923417 0.1076583]
Row 2001 -> Prediction: Good Loan, Probability (Bad/Good): [0.34374569 0.65625431]
Row 2002 -> Prediction: Good Loan, Probability (Bad/Good): [0.4650773 0.5349227]


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values